In [ ]:
import numpy as np
from matplotlib import pyplot,colors
from __future__ import division
from scipy.optimize import curve_fit
%matplotlib inline

## Se definen algunas de las funciones a usar.

In [ ]:
# Magnetizacion de todo el sistema
def Magneto(S):  
   #Suma todos los elementos de la matriz. 
    M = np.sum(S)
    return M

In [ ]:
# Energia de todo el sistema
def EnergiaTotal(S,L): 
    E = 0

    for i in range(0,L-1):
        for j in range(0,L-1):
            E += -B*S[i,j]-J*S[i,j]*(S[i-1,j]+S[i,j-1]+S[i,(j+1)%L]+S[(i+1)%L,j])/2
            # (j+1)%L y (i+1)%L arregla el problema de los bordes. Si i+1<L => (i+1)%L=i+1. En el caso
            # en que i+1 = L devuelve 0. De esa manera me aseguro que siempre me quede dentro del tamaño del array.
    return E

In [ ]:
#Me quedo con los estados mas probables
def isingin2D(S, beta, B,L):
    # Elegimos una posición del array al azar.
    i = np.random.randint(0,L) #Crea un entero aleatorio entre 0 y L-1, osea L elementos.
    j = np.random.randint(0,L) 

    # Veo la energia del estado inicial:
    Einicial = -B*S[i,j]-J*S[i,j]*(S[i-1,j]+S[i,j-1]+S[i,(j+1)%L]+S[(i+1)%L,j]) # antes de rotar el espin
    deltaE = -2*Einicial # Diferencia de energia entre tener spin up or down en el punto (i,j)
    #Al estar el spin de dicho punto factorizado, la diferencia es dos veces la Energia inicial

    
    #Si delta E<0, entonces invertimos el espin, por lo tanto aparece un cambio en la energia
    if deltaE <= 0:
        # Cambia el spin:
        dE = deltaE
        dM = -2*S[i,j]
        S[i,j] = -S[i,j]

    else:
        #Acá está es la división de las probabilidades de transición. Está es la condición de balance detallado a su vez
        # p es lo que sacamos del cociente de probabilidad de transición a el estado spin up sobre el estado spin down
 
        p = np.exp(-beta*deltaE)
        r = np.random.random_sample() #Esta funcion retorna numeros entre 0 y 1 aleatoriamente
        #La p es el cociente de probabilidades de transición, y a su vez cumple que es una distribución normalizada a 1
        #nos dara cual de los dos estados es más probable. De esta manera no tenemos que sacar la cte de normalizacion
        if r < p:
            #Cambia el spin:
            dE = deltaE
            dM = -2*S[i,j]
            S[i,j] = -S[i,j]
        else:
            #No cambia el spin, entonces todo aporta 0.
            dE = 0
            dM = 0
   
    return S, dE, dM  # Devuelve los nuevos S, dE, dM

In [ ]:
def functionTermalizadora(L,S,beta,B,J,npre,npasos,estadoinicial): #Estadoinicial = 1 me plotea el S incial
    energia = np.zeros(npasos)
    magnet = np.zeros(npasos)

    for n in range(npre):
        S, dE, dM=isingin2D(S,beta,B,L)
        
    # muestro el estado inicial (Simplemente grafica la red inicial, si pongo en el ultimo parametro 0 esto lo saltea)
    if estadoinicial == 1:
        pyplot.figure(1)
        cmap = colors.ListedColormap(['white', 'red'])
        pyplot.imshow(S,interpolation='none',cmap=cmap)
        pyplot.title("Estado inicial de la matriz:")
        pyplot.show(block=False)
    
    #El estado inicial de energia y magnetización
    energia[0] = EnergiaTotal(S,L)
    magnet[0] = Magneto(S)

    for n in range(npasos-1):
        S, dE, dM = isingin2D(S,beta,B,L);
        energia[n+1] = energia[n] + dE;
        magnet[n+1] = magnet[n] + dM;


    return energia, magnet 

## Luego de algunas pruebas se vio que la termalizacion depende de la temperatura y del tamaño del arreglo 

In [ ]:

pyplot.figure(2)
#pyplot.plot(magnetT0/(L0*L0),label='L = 8') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT1/(L1*L1),label='L = 16') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT2/(L2*L2),label='L = 32') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT3/(L3*L3),label='L = 64') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT4/(L4*L4),label='L = 128') #(Vemos la magnetizacion por celda)
pyplot.title("Magnetizacionn del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Magnetizacion por sitio')
pyplot.legend()
pyplot.show()

pyplot.figure(3)
#pyplot.plot(energiaT0/(L0*L0),label='L = 8')
pyplot.plot(energiaT1/(L1*L1),label='L = 16')
pyplot.plot(energiaT2/(L2*L2),label='L = 32')
pyplot.plot(energiaT3/(L3*L3),label='L = 64')
pyplot.plot(energiaT4/(L4*L4),label='L = 128')
pyplot.title("Energia del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Energia por sitio')
pyplot.legend()
pyplot.show()

In [ ]:
L0=8
L1 = 16 #tamaño de un lado de la red de spines
L2= 32
L3 = 64 
L4= 128
T = 1
beta = 1/T #MU=1 por conveniencia
J = 1
B = 0 #Este es le campo magnetico que está apagado, hay que prenderlo y tirar valores a ver si termina dependiendo del B o no.
S = 2*(np.random.rand(L0,L0)>0.5) -1; # Genero un estado aleatorio de espines(1,-1) en un arreglo de L por L
#Tira un numero entre 0 y 1, si es mayor a 0.5 le asigna 1.
#Si es menor le asigna -1. Luego 1*2-1=1 y 0*2-1=-1

npre = 0
npasos = 100000#Cantidad de pasos posteriores
energia= np.zeros(npasos)
magnet = np.zeros(npasos)
energiaT0, magnetT0= functionTermalizadora(L0,S,beta,B,J,npre, npasos,1)

S = 2*(np.random.rand(L1,L1)>0.5) -1;
energia= np.zeros(npasos)
magnet = np.zeros(npasos)
energiaT1, magnetT1= functionTermalizadora(L1,S,beta,B,J,npre, npasos,1)

S = 2*(np.random.rand(L2,L2)>0.5) -1;
energia= np.zeros(npasos)
magnet = np.zeros(npasos)
energiaT2, magnetT2= functionTermalizadora(L2,S,beta,B,J,npre, npasos,1)
S = 2*(np.random.rand(L3,L3)>0.5) -1;
energia= np.zeros(npasos)
magnet = np.zeros(npasos)
energiaT3, magnetT3= functionTermalizadora(L3,S,beta,B,J,npre, npasos,1)
S = 2*(np.random.rand(L4,L4)>0.5) -1;
energia= np.zeros(npasos)
magnet = np.zeros(npasos)
energiaT4, magnetT4= functionTermalizadora(L4,S,beta,B,J,npre, npasos,1)

pyplot.figure(2)
#pyplot.plot(magnetT0/(L0*L0),label='L = 8') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT1/(L1*L1),label='L = 16') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT2/(L2*L2),label='L = 32') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT3/(L3*L3),label='L = 64') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT4/(L4*L4),label='L = 128') #(Vemos la magnetizacion por celda)
pyplot.title("Magnetizacionn del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Magnetizacion por sitio')
pyplot.legend()
pyplot.show()

pyplot.figure(3)
#pyplot.plot(energiaT0/(L0*L0),label='L = 8')
pyplot.plot(energiaT1/(L1*L1),label='L = 16')
pyplot.plot(energiaT2/(L2*L2),label='L = 32')
pyplot.plot(energiaT3/(L3*L3),label='L = 64')
pyplot.plot(energiaT4/(L4*L4),label='L = 128')
pyplot.title("Energia del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Energia por sitio')
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.figure(2)

pyplot.plot(magnetT1/(L1*L1),label='L = 16') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT2/(L2*L2),label='L = 32') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT3/(L3*L3),label='L = 64') #(Vemos la magnetizacion por celda)
pyplot.plot(magnetT4/(L4*L4),label='L = 128') #(Vemos la magnetizacion por celda)
pyplot.title("Magnetizacionn del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Magnetizacion por sitio')
pyplot.legend()
pyplot.show()

pyplot.figure(3)

pyplot.plot(energiaT1,label='L = 16')
pyplot.plot(energiaT2,label='L = 32')
pyplot.plot(energiaT3,label='L = 64')
pyplot.plot(energiaT4,label='L = 128')
pyplot.title("Energia del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Energia por sitio')
pyplot.legend()
pyplot.show()

In [ ]:
L = 8 #tamaño de un lado de la red de spines
T = 2.2
beta = 1/T #MU=1 por conveniencia
J = 1
B = 0 #Este es le campo magnetico que está apagado, hay que prenderlo y tirar valores a ver si termina dependiendo del B o no.
S = 2*(np.random.rand(L,L)>0.5) -1; # Genero un estado aleatorio de espines(1,-1) en un arreglo de L por L
#Tira un numero entre 0 y 1, si es mayor a 0.5 le asigna 1.
#Si es menor le asigna -1. Luego 1*2-1=1 y 0*2-1=-1
cantidadTemperaturas = 8

for i in range(0,L,16):
    for i in range(0,cantidadTemperaturas,2):
        beta = 1/i 
        npre = 10000
        npasos = 1000000#Cantidad de pasos posteriores
        energia= np.zeros(npasos)
        magnet = np.zeros(npasos)

        energia, magnet= functionTermalizadora(L,S,beta,B,J,npre, npasos,1)

pyplot.figure(2)
pyplot.plot(magnet/(L*L)) #(Vemos la magnetizacion por celda)
pyplot.title("Magnetizacionn del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Magnetizacion por sitio')
pyplot.show()

pyplot.figure(3)
pyplot.plot(energia/(L*L))
pyplot.title("Energia del sistema:")
pyplot.xlabel('Paso')
pyplot.ylabel('Energia por sitio')
pyplot.show()

## Se puede observar en las figuras de arriba que en estas condiciones tanto la energia y la magnetizacion del sistema logran llegar a un equilibrio despues de las $10^{4}$ iteraciones

## Defino las funciones para calcular la media de la magnetizacion y la energia. Ademas la dispercion cuadratica.

In [ ]:
def CalcMedio(magnet, energia):  # magnetizacion de todo el sistema
   #Suma todos los elementos de la matriz.
    npasos=np.size(magnet)
    difn=30000
    ni = npasos - difn
    MM = 0
    EM = 0
    
    for i in range(ni, npasos - 1):
        MM += magnet[i]/difn
        EM += energia[i]/difn

    return MM,EM

In [ ]:
def Dispersion(magnet, energia, temperatura):
    MM,EM = CalcMedio(magnet, energia)
    npasos=np.size(magnet)
    difn=30000
    ni = npasos - difn
    MM2 = 0
    EM2 = 0
    dispeM = 0
    dispeE = 0
    sup=0
    cv=0
    
    for i in range(ni,npasos-1):
        MM2 += magnet[i]*magnet[i]/difn
        EM2 += energia[i]*energia[i]/difn
    MM2 = MM2/npasos
    EM2 = EM2/npasos
    dispeM = np.sqrt(MM2-MM*MM)
    dispeE = np.sqrt(EM2-EM*EM)
    sup = (MM2-MM*MM)*temperatura
    cv=(EM2-EM*EM)*(temperatura*temperatura)
    return dispeM, dispeE,sup,cv

## Se define una funcion para poder calcular a varias temperaturas y tomar los valores medios y disperciones cuadraticas

In [ ]:
def MagnetEnergCvSucep(L,B,J,cantidadTemperaturas,Temp): #Hago el barrido de temperaturas, utilizando para cada una de
                                                        #ellas la funcion termalizadora.
    
    magnetizacionmediadeT = np.zeros(cantidadTemperaturas)
    DispM = np.zeros(cantidadTemperaturas)
    energiamediadeT = np.zeros(cantidadTemperaturas)
    temperatura = np.zeros(cantidadTemperaturas)
    DispE = np.zeros(cantidadTemperaturas)
    sup = np.zeros(cantidadTemperaturas)
    cv = np.zeros(cantidadTemperaturas)
    npre = 10000
    npasos = 1000000
    
    for i in range(0,cantidadTemperaturas):
        print(i)      
        temperatura[i]=Temp[i]
        beta = 1.0/(temperatura[i])
        energia= np.zeros(npasos)
        magnet = np.zeros(npasos)
        energia, magnet = functionTermalizadora(L,S,beta,B,J,npre, npasos,0)
        npre=0 #solo quiero que pretermalize la primera vez
        magnetizacionmediadeT[i], energiamediadeT[i] = CalcMedio(magnet, energia)
        DispM[i], DispE[i], sup[i], cv[i]= Dispersion(magnet, energia, temperatura[i])
        
    return magnetizacionmediadeT, temperatura, DispM, DispE, energiamediadeT,sup,cv

## Se quiere ver el cambio de fase.

In [ ]:
Temp1 = np.linspace(0.1,1.5,50)
Temp2 = np.linspace(1.5,3,200)
Temp3 = np.linspace(3,4,50)

Temp = np.zeros(300) #una forma a la fuerza bruta para generarme el vector que quiero de temperaturas.
for i in range(len(Temp)):
    if i<50:
        Temp[i]=Temp1[i]
    elif i>=50:
        if i <250:
            Temp[i]=Temp2[i-50]
        elif i>=250:
            Temp[i] = Temp3[i-250]

In [ ]:
cantidadTemperaturas = len(Temp)
B=0
J=1.0
L0=16
L1=32
L2=64
S = np.ones((L0,L0)); 
#S = 2*(np.random.rand(L,L)>0.5) -1;
magnetizacionmediadeT16, temperatura, DispM, DispE, energiamediadeT16,sup,cv = MagnetEnergCvSucep(L0,B,J,cantidadTemperaturas,Temp)
S = np.ones((L1,L1)); 
magnetizacionmediadeT32, temperatura, DispM, DispE, energiamediadeT32,sup,cv = MagnetEnergCvSucep(L1,B,J,cantidadTemperaturas,Temp)
S = np.ones((L2,L2)); 
magnetizacionmediadeT64, temperatura, DispM, DispE, energiamediadeT64,sup,cv = MagnetEnergCvSucep(L2,B,J,cantidadTemperaturas,Temp)
L4=4
L8=8

S = np.ones((L8,L8)); 
#S = 2*(np.random.rand(L,L)>0.5) -1;
magnetizacionmediadeT8, temperatura, DispM, DispE, energiamediadeT8,sup,cv = MagnetEnergCvSucep(L8,B,J,cantidadTemperaturas,Temp)
S = np.ones((L4,L4)); 
magnetizacionmediadeT4, temperatura, DispM, DispE, energiamediadeT4,sup,cv = MagnetEnergCvSucep(L4,B,J,cantidadTemperaturas,Temp)


In [ ]:
cantidadTemperaturas = len(Temp)
B=0
J=1.0
L4=4
L8=8

S = np.ones((L8,L8)); 
#S = 2*(np.random.rand(L,L)>0.5) -1;
magnetizacionmediadeT8, temperatura, DispM, DispE, energiamediadeT8,sup,cv = MagnetEnergCvSucep(L8,B,J,cantidadTemperaturas,Temp)
S = np.ones((L4,L4)); 
magnetizacionmediadeT4, temperatura, DispM, DispE, energiamediadeT4,sup,cv = MagnetEnergCvSucep(L4,B,J,cantidadTemperaturas,Temp)


In [ ]:
L5=32
S = np.ones((L5,L5)); 
magnetizacionmediadeT, temperatura, DispM, DispE, energiamediadeT,sup,cv = MagnetEnergCvSucep(L5,B,J,cantidadTemperaturas,Temp)


In [ ]:
pyplot.figure(4)
#pyplot.plot(temperatura,energiamediadeT4/(L4*L4),label='L = 4')
pyplot.scatter(temperatura,magnetizacionmediadeT8/(L8*L8),label='L = 8')
pyplot.scatter(temperatura,magnetizacionmediadeT16/(L0*L0),label='L = 16')
pyplot.scatter(temperatura,magnetizacionmediadeT32/(L1*L1),label='L = 32')
pyplot.scatter(temperatura,magnetizacionmediadeT64/(L2*L2),label='L = 64')
pyplot.scatter(temperatura,magnetizacionmediadeT128/(L5*L5),label='L = 128')
pyplot.xlabel('Temperatura')
pyplot.ylabel('Magnetizacion Media por sitio')
pyplot.grid()
pyplot.legend()
pyplot.show()

pyplot.figure(5)

#pyplot.plot(temperatura,energiamediadeT4/(L4*L4),label='L = 4')
pyplot.scatter(temperatura,energiamediadeT8/(L8*L8),label='L = 8')
pyplot.scatter(temperatura,energiamediadeT16/(L0*L0),label='L = 16')
pyplot.scatter(temperatura,energiamediadeT32/(L1*L1),label='L = 32')
pyplot.scatter(temperatura,energiamediadeT64/(L2*L2),label='L = 64')
pyplot.scatter(temperatura,energiamediadeT128/(L5*L5),label='L = 128')
pyplot.xlim(1,4)
pyplot.xlabel('Temperatura')
pyplot.ylabel('Energia Media por sitio')
pyplot.grid()
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.figure(4)
pyplot.plot(temperatura,magnetizacionmediadeT128/(L*L))

pyplot.xlabel('Temperatura')
pyplot.ylabel('Magnetizacion Media por sitio')
pyplot.grid()
pyplot.show()

pyplot.figure(5)
pyplot.plot(temperatura,energiamediadeT128/(L*L))

pyplot.xlabel('Temperatura')
pyplot.ylabel('Energia Media por sitio')
pyplot.grid()
pyplot.show()

pyplot.figure(6)
pyplot.plot(temperatura,-sup) 
pyplot.xlim(0,4)

pyplot.xlabel('Temperatura')
pyplot.ylabel('Suceptibilidad')
pyplot.grid()
pyplot.show()



pyplot.figure(8)
pyplot.plot(temperatura,-cv/(L5*L5))
pyplot.xlim(0.1,4)
pyplot.xlabel('Temperatura')
pyplot.ylabel('Cv')
pyplot.grid()
pyplot.show()



In [ ]:
cv

# Se puede observar que cerca de la temperatura critica,  la cual teoricamente es T=2.25 Kelvin, empieza el cambio de fase y es donde están los saltos o los picos en los graficos.

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm 
temp=temperatura
mag=magnetizacionmediadeT128/(L*L)

pyplot.scatter(temp,mag)
pyplot.scatter(temp[125:158],smooth(mag[125:158],5))
temp = temp[125:158]
mag= smooth(mag[125:158],5)



In [ ]:
pyplot.scatter(temp[2:31],mag[2:31])

In [ ]:
def smooth(y, box_pts): 
    box = np.ones(box_pts)/box_pts 
    y_smooth = np.convolve(y, box, mode='same') 
    return y_smooth 

In [ ]:

mag_log = np.log(mag[2:28])
temp_log = [np.log((2.28 - t)) for t in temp[2:28]]
#plt.plot(temp_log[10:35],mag_log[10:35])

In [ ]:
from scipy.stats import linregress
slope, intercept, r_value, p_value, std_err = linregress(temp_log,mag_log)
ajuste = [intercept + slope*t for t in temp_log]

In [ ]:
plt.figure()
plt.plot(temp_log,mag_log,color='blue',label='Simulación')
plt.plot(temp_log,ajuste, color='red',label='Regresión lineal: pendiente {:.3f}'.format(slope))
plt.xlabel('log(2.28-T)')
plt.ylabel('log(m)')
plt.legend()
plt.show()